In [1]:
import numpy as np
import random as rd
import pylab
import matplotlib
import matplotlib.pyplot as plt

import math
import time

In [2]:
#pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
from gurobipy import *

In [4]:
#Lagrangian Dual Maximization

def maximize_L(training_set):
    m = Model("Dual Lagrangian")
    vars = ['']*int(round(element_num*training_rate))
    for i in range(int(round(element_num*training_rate))):
        vars[i] = m.addVar(vtype=GRB.CONTINUOUS, name="x"+str(i))
    m.update()

    ## Objective function 구성
    obj_1 = sum(vars[i]*inner_product(training_set[i], training_set[i]) for i in range(int(round(element_num*training_rate))))
    obj_2 = 0
    for i in range(int(round(element_num*training_rate))):
        for j in range(int(round(element_num*training_rate))):
            obj_2 += vars[i]*vars[j]*inner_product(training_set[i], training_set[j])
    obj = obj_1 - obj_2

    m.setObjective(obj, GRB.MAXIMIZE)
    for i in range(int(round(element_num*training_rate))):
        m.addConstr(vars[i] <= C, "c"+str(i))
    m.optimize()

    ## Rounding up
    solution = []
    for v in m.getVars():
        solution.append(round(v.x, int(math.floor(abs(np.log10(C))+2))))
    return solution

In [5]:
# alpha 값에 따라 데이터를 분류

def select_sv_on(alpha_dict):
    sv_on_point_index = []
    for i in range(int(round(element_num*training_rate))):
        if alpha_dict[i] > 0:
            if alpha_dict[i] < C:
                sv_on_point_index.append(i)
    print("number sv_on: "+str(len(sv_on_point_index)))
    return sv_on_point_index

def select_sv_outsider(alpha_dict):
    sv_outsider_point_index = []
    for i in range(int(round(element_num*training_rate))):
        if alpha_dict[i] == C :
            sv_outsider_point_index.append(i)
    print("number sv_outsider: "+str(len(sv_outsider_point_index)))
    return sv_outsider_point_index

def select_nsv(sv_on, sv_outsider):
    index = [i for i in range(int(round(element_num*training_rate)))]
    sv = sv_on + sv_outsider
    for i in sv:
        index.remove(i)
    print("number nsv: "+str(len(index)))
    return index

In [6]:
# Radius 계산

def calculate_radius(datapoint, point_dict, alpha_dict, middlepoint):
    R_1 = inner_product(datapoint, datapoint)
    R_2 = sum([alpha_dict[i]*inner_product(point_dict[i], datapoint) for i in range(training_num)])
    R_3 = 0
    for i in range(training_num):
        for j in range(training_num):
            R_3 += alpha_dict[i]*alpha_dict[j]*inner_product(point_dict[i], point_dict[j])
    R =(R_1 - 2*R_2 + R_3)**0.5
    return R

In [7]:
# Outlier test(Yes/No)

def test_SVDD(test_set, data_class, middlepoint, R, point_dict, alpha_dict):
    FRR2, FAR2 = 0, 0
    for i in range(len(test_set)):
        if data_class[i] == 1:
            FRR2 +=1
        else:
            FAR2 +=1
    i, FRR1, FAR1 = 0, 0, 0
    for test_data in test_set:
        if calculate_radius(test_data, point_dict, alpha_dict, middlepoint) > R :
            print("Data #" + str(i+1) + " : No")
            if data_class[i] == 1:
                FRR1 += 1
        else:
            print("Data #" + str(i+1) + " : Yes")
            if data_class[i] == 2:
                FAR1 += 1
        i+=1
    print("\nTotal set : " + str(len(test_set)))
    print("FRR : " + str(FRR1/float(FRR2)))
    print("FAR : " + str(FAR1/float(FAR2)))
    print("Radius R : " + str(R))
    return FRR1/float(FRR2), FAR1/float(FAR2)